# BLU15 - Model CSI
## Part 1 of 2 - When to train your model

In this notebook we will be covering the following:

- 1. The need for retraining
  - 1.1 Data drift
  - 1.2 Robustness
  - 1.3 When ground truth is not available at the time of model training
  - 1.4 Concept drift
- 2. How to measure the decline in model performance?
  - 2.1 Histogram
  - 2.2 K-S Statistic
  - 2.3 Target distribution
  - 2.4 Correlation
- 3. Retraining strategy
- 4. How much data is needed for retraining?
  - 4.1 Fixed window size
  - 4.2 Dynamic window size
  - 4.3 Combining all of the data
- 5. Final considerations

## 1. The need for retraining

*Train, test and deploy* – that’s it, right? Is your work done? **Not quite!**

So far this is how your process has been going:

1. Assuming the sufficient historical data available, model building starts by learning the dependencies between a set of independent features and the target variable.
2. The best learnt dependency is calculated basis some evaluation metric to minimize the predictions error on the validation dataset
3. This best learnt model is then deployed in production with the **expectation** to keep making accurate predictions on incoming unseen data **for as long as possible**

One of the biggest mistake a data scientist can make is assume their models will keep working properly forever after deployment. *But what about the data, which will inevitably keep changing?* 

A model deployed in production and left to itself won’t be able to adapt to changes in data by itself.

Let's look at the following example:
<img src="https://i1.wp.com/neptune.ai/wp-content/uploads/Retraining-models-impact-of-changes.png?resize=829%2C354&ssl=1" alt="drift" width="800"/>

>In a [UK bank survey from August 2020](https://www.bankofengland.co.uk/bank-overground/2021/how-has-covid-affected-the-performance-of-machine-learning-models-used-by-uk-banks), 35% of asked bankers reported a negative impact on ML model performance because of the pandemic. 

Unpredictable events like this are a great example of why continuous training and monitoring of ML models in production is important compared to static validation and testing techniques. 

Ideally, **retraining involves running the entire existing pipeline with new data**. That’s it. It does not involve any code changes or re-building the pipeline. However, if you end up exploring a new algorithm or a feature which might not have been available at the time of previous model training, then incorporating these changes while deploying the retrained model will further improve the model accuracy.

But what exactly can cause this decrease in performance?

### 1.1 Data drift

To understand this, let us recall one of the most critical assumptions in ML modelling 

> The train and test dataset should belong to similar distribution. 

The model will perform well if the new data is similar to the data observed in the past on which the model was trained on. Therefore, it's understandable that **if test data distribution deviates from that of train data, the model will not hold well**. 

There's many factors that can cause such deviation, depending on the business case, e.g. change in consumer preferences, fast moving competitive space, geographic shift, economic conditions, **a pandemic**, etc. Hence, the **drifting data distribution calls for an ongoing process of periodically checking the validity of old model**. In short, it is critical to keep your machine learning model updated; but the key is when? We will discuss this in a bit...

### 1.2 Robustness

As you remember from [SLU17 - Ethics and Fairness](https://github.com/LDSSA/batch5-students/tree/main/S01%20-%20Bootcamp%20and%20Binary%20Classification/SLU17%20-%20Ethics%20and%20Fairness), a model has an impact in the world that it learned from. And that impact can change the *a priori* assumptions that once were true. 

People/entities that get affected by the outcome of the ML models may deliberately **alter their response** in order to send spurious input to the model, thereby **escaping the impact of the model predictions**. For example, the models such as *fraud detection* and *cyber-security* receive manipulated and distorted inputs which cause model to output misclassified predictions. Such type of adversaries also drives down the model performance.


### 1.3 When ground truth is not available at the time of model training

In most of the ML models, **the ground truth labels are not available to train the model**. For example, target variable which captures the response of the end user is not known. In that case, your best bet could be to **mock the user action based on certain set of rules coming from business understanding** or **leverage the open source dataset** to initiate model training. But, this model might not necessarily represent the actual data and hence will not perform well until a burn-in period where it starts picking (aka learning) the true actions of the end user.

### 1.4 Concept drift

Concept drift is a phenomenon where **the meaning the labels of the target variable you’re trying to predict changes over time. This means that the concept has changed but the model doesn’t know about the change**. 

Concept drift happens when **the original idea your model had about the target class changes**. For example, you build a model to classify positive and negative sentiment of tweets around certain topics, and over time people’s sentiment about these topics changes. Tweets belonging to positive sentiment may evolve over time to be negative. 

In simple terms, the concept of sentiment analysis has drifted. Unfortunately, your model will keep predicting positive sentiments as negative sentiments.

## 2. How to measure the decline in model performance?

If **the ground truth values are stored alongside the predictions**, such as with the success of a search, the decline (or not) is calculated on a **continuous basis to assess the drift**.


<img src="https://static.tildacdn.com/tild3462-6534-4732-a462-643534313536/model_decay_retraini.png" alt="retraining" width="500"/>

**But what if the prediction horizon is farther into the future and we can’t wait till the ground truth label is observed to assess the model goodness?** Well, in that case, **we can roughly estimate the retraining window from back-testing**. This involves using the ground truth labels and predictions from the historical data to estimate the time frame around which the accuracy begins to taper off.

>Effectively, the whole exercise of finding the model drift boils down to inferring whether the two data sets (training and test) are coming from the same distribution, or if the performance has fallen below acceptable range.

Lets look at some of the ways to assess the distribution drift:

### 2.1 Histogram

A quick way to visualize the comparison is to draw the histogram — the degree of overlap between the two histograms gives a measure of similarity.
<img src="https://miro.medium.com/max/1400/1*Q4tXoLAbIRonpGNxdADVlA.png" alt="histogram" width="700"/>


### 2.2 K-S statistic

The [Kolmogorov–Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) is a useful tool to check if the upcoming new data belongs to the same distribution as that of training data. In short, this test quantifies a distance between the empirical distribution function of the sample and the cumulative distribution function of the reference distribution, or between the empirical distribution functions of two samples. Python has an implementation of this test provided by [SciPy](https://scipy.org/) by Statistical functions ([scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html)).

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cf/KS_Example.png" alt="ks" width="500"/>
<center>llustration of the Kolmogorov–Smirnov statistic. The red line is a model cumulative distribution function, the blue line is an empirical cumulative distribution function, and the black arrow is the K–S statistic.</center>


### 2.3 Target distribution

One quick way to check the consistent predictive power of the ML model is to **examine the distribution of the target variable**. For example, if your training dataset is imbalanced with 99% data belonging to class 1 and remaining 1% to class 0. And, the predictions reflect this distribution to be around 90%-10%, then it should be treated as an alert for further investigation.

<img src="https://www.guru99.com/images/r_programming/032918_0752_BarChartHis7.png" alt="histogram" width="500"/>

### 2.4 Correlation

Additionally, monitoring pairwise correlations between individual predictors will help bring out an underlying drift.

## 3. Retraining strategy

There's two approaches to handling model retraining:

- **Model is retrained at a fixed periodic interval**
The model is retrained at a set interval. If the incoming data is changing frequently, the model retraining can happen even daily! A fixed preriod to retrain doesn't necessarily mean a less frequent.
  

- **Model is continuously retrained**
  - **Trigger based on performance metrics**
The model is retrained when a trigger is activated by monitoring the performance metrics. This approach is more effective than the above but the threshold specifying the acceptable level of performance divergence needs to be decided to initiate retraining. The following factors need to be considered while deciding the threshold:
    - Too low a threshold will lead to frequent retraining which will lead to increased overhead in terms of compute cost
    - Too high a threshold will output “strayed predictions”
<img src="https://i1.wp.com/neptune.ai/wp-content/uploads/Retraining-models-graph.jpg?resize=768%2C645&ssl=1" alt="histogram" width="500"/>
  - **Trigger based on data changes**
By monitoring your upstream data in production, you can identify changes in the distribution of your data. This can indicate that your model is outdated or that you’re in a dynamic environment. It’s a good approach to consider when you don’t get quick feedback or ground truth from your model in production.
<img src="https://miro.medium.com/max/1248/1*aAR12f8rwroVf0O6Z1ohcw.png" alt="histogram" width="500"/>
<center>Example of a retraining stategy based on data changes identified by monitoring upstream data in production.</center>
- **Retraining on demand**
Of all the options, this is the less efficient as it does not rely on automation, but it's the most simple to implement and therefore, sometimes favoured instead of the others. This is the approach that we will follow in the next `Learning notebook`. 


## 4. How much data is needed for retraining?

In addition to knowing why and when you need to retrain your models, it’s also important to know how to select the right data for retraining, and whether or not to drop the old data. 

Three things to consider when choosing the right size of data:

- What is the size of your data?
- Is your data drifting?
- How often do you get new data?

### 4.1 Fixed window size

This is a straightforward approach to selecting the training data. **Selecting the right window size** is a major drawback to using this approach:

- If the **window size is too large, we may introduce noise into the data**. 
- If it’s **too narrow, it might lead to underfitting**.

Overall, this approach is a simple heuristic approach that will work well in some cases, but will fail in a dynamic environment where data is constantly changing.

### 4.2 Dynamic window size

This is an alternative to the fixed window size approach. This approach **helps to determine how much historical data should be used to retrain your model by iterating through the window size to determine the optimal value to use**. It’s an approach to consider if your data is large and you also get new data frequently. 

<img src="https://i2.wp.com/neptune.ai/wp-content/uploads/Training-data-vs-test-data.png?resize=900%2C420&ssl=1" alt="histogram" width="700"/>


### 4.3 Combining all of the data

The simplest way, resources permitting, to handle this problem is simply to combine all of the data and retrain your model. As you're keeping data where a change has been detected, this is more sensitive to another drift and will need to be monitored closely. In production this is usually not a viable option due to the computational requirements needed as data continues to grow.

## 5. Final considerations

Before we move on to a more practical demonstration, I hope you're now aware that retraining and redeployment is a constant need in any ML model. The *when* and the *how* are key questions that rely on the sensitivity of not only the methods, but of the Data Scientists itself. As *Scientists*, a critical view of any result is a fundamental skill. Now let's get our hands dirty in Part 2! 